# 导入包

In [3]:
!pip install modlamp
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.0 MB 5.3 MB/s 
     |████████████████████████████████| 154 kB 44.5 MB/s 
     |████████████████████████████████| 345 kB 65.4 MB/s 
  Created wheel for modlamp: filename=modlamp-4.3.0-py3-none-any.whl size=173903 sha256=6f1221e5c3456ad28d604b5d9bdec35a48d9b1ebe346a5d595169cfae064b161
  Stored in directory: /root/.cache/pip/wheels/c4/d6/6d/6eadb0c0a04a2f203b9bb193a050e787f218a9a6d8add543dd
Successfully built modlamp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.3 MB 1.8 MB/s 


In [4]:
cd /content/drive/MyDrive/Peptide half-life

/content/drive/MyDrive/Peptide half-life


In [5]:
import pandas as pd
from utils import *
from datasets import Dataset
from descriptors import getMoleculeDesc,getMoleculeAndPeptideDesc,getAllDesc
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn import feature_selection, pipeline, preprocessing, model_selection, metrics
import numpy as np
import joblib
from copy import deepcopy
import numpy as np
import time

In [6]:
def readFileAndCal(path,seqField='SEQUENCE',smiField='SMILES',labelField='T',mode='getMoleculeDesc',before_num=50,before_open=True):
    #计算前before_num个数据
    data=pd.read_excel(path)

    if mode=='getAllDesc':
        if before_open:
            desc=getAllDesc(seqToUpper(list(data[seqField]))[:before_num],list(data[smiField])[:before_num])
        else:
            desc=getAllDesc(seqToUpper(list(data[seqField])),list(data[smiField]))
    elif mode=='getMoleculeAndPeptideDesc':
        if before_open:
            desc=getMoleculeAndPeptideDesc(seqToUpper(list(data[seqField]))[:before_num],list(data[smiField])[:before_num])
        else:
            desc=getMoleculeAndPeptideDesc(seqToUpper(list(data[seqField])),list(data[smiField]))
    elif mode=='getMoleculeDesc':
        if before_open:
            desc=getMoleculeDesc(seqToUpper(list(data[seqField]))[:before_num],list(data[smiField])[:before_num])
        else:
            desc=getMoleculeDesc(seqToUpper(list(data[seqField])),list(data[smiField]))

    featureName=list(desc.keys())

    df=pd.DataFrame.from_dict(desc)
    df['label']=data[labelField]
    
    return df,featureName

In [7]:
dataPath='/content/drive/MyDrive/Peptide half-life/data/'
featurePath='/content/drive/MyDrive/Peptide half-life/features/'
modelPath='/content/drive/MyDrive/Peptide half-life/model/'

# Human blood (modification)

In [8]:
start = time.perf_counter()

In [9]:
path='Human blood (modification)'

In [ ]:
Human_blood_modification,featureName=readFileAndCal(path=dataPath+path+'.xlsx',before_open=True)

In [ ]:
Human_blood_modification

In [ ]:
end = time.perf_counter()
print("目前运行时间为", round(end-start), 'seconds')

In [11]:
featureName[:10]

['amidation',
 'acetylation',
 'modlAMPLength',
 'modlAMPMW',
 'modlAMPCharge',
 'modlAMPChargeDensity',
 'modlAMPpI',
 'modlAMPInstabilityInd',
 'modlAMPAromaticity',
 'modlAMPAliphaticInd']

In [12]:
Human_blood_modification.to_csv(featurePath+path+'.csv')

In [13]:
peptide_len=Human_blood_modification['modlAMPLength']
max_len=max(peptide_len)
min_len=min(peptide_len)
max_len,min_len

(39.0, 4.0)

In [14]:
Human_blood_modification=Dataset(Human_blood_modification,featureName,['label'],cut_len=0, max_len=max_len, min_len=min_len)
x_train, y_train, x_test, y_test=Human_blood_modification.getData()

length_count:5.0     8
6.0     7
8.0     7
38.0    6
10.0    5
27.0    4
4.0     2
29.0    2
9.0     2
13.0    1
39.0    1
7.0     1
11.0    1
28.0    1
36.0    1
37.0    1
Name: modlAMPLength, dtype: int64


In [15]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((43, 765), (43, 1), (7, 765), (7, 1))

In [16]:
x_train[:5],y_train[:5],y_test

(array([[1.   , 0.   , 6.   , ..., 0.082, 0.109, 0.182],
        [0.   , 0.   , 6.   , ..., 0.079, 0.124, 0.174],
        [0.   , 1.   , 6.   , ..., 0.078, 0.101, 0.155],
        [0.   , 1.   , 6.   , ..., 0.073, 0.104, 0.146],
        [1.   , 1.   , 6.   , ..., 0.073, 0.104, 0.146]], dtype=float32),
 array([[1800.],
        [1800.],
        [3600.],
        [3600.],
        [5400.]], dtype=float32),
 array([[ 5400.],
        [ 7200.],
        [ 1080.],
        [86400.],
        [86400.],
        [  900.],
        [72000.]], dtype=float32))

In [ ]:
def myMetrics(y_true,y_pred):

    mae=metrics.mean_absolute_error(y_true,y_pred)

    mse=metrics.mean_squared_error(y_true,y_pred,squared=False)

    rmse=metrics.mean_squared_error(y_true,y_pred,squared=True)

    r2=metrics.r2_score(y_true, y_pred)

    r=np.corrcoef(y_true, y_pred)

    return r,r2,mae,mse,rmse

In [18]:
def myElasticNet(x_train, y_train, x_test, y_test, featureName, k_min, k_step, model_file, grid_search_file,
				 random_state=0):
	regr = ElasticNet(random_state=random_state)
	selector = feature_selection.SelectKBest()
	
	# 特征处理和模型训练统一,避免测试集和训练集分别进行预处理,防止将训练集信息引入测试集
	estimator = pipeline.Pipeline([
		('stand_scaler', preprocessing.StandardScaler()),  # 弹性网络有惩罚项,必须进行标准化
		('minmax_scaler', preprocessing.MinMaxScaler()),
		# https://blog.csdn.net/ccblogger/article/details/81866631提到线性回归类的问题需要最优化
		('selector', selector),
		('regr', regr),  # 包含超参数alpha(惩罚项),l1_ratio(调整l1和l2范数的比例)
	])
	
	k_grid = list(np.arange(k_min, len(featureName), k_step))  # 挑选最优的k个特征,selector的超参数
	
	score_func_grid = [feature_selection.f_regression, feature_selection.mutual_info_regression]  # 挑选特征的方法,selector的超参数
	
	alpha_grid = [0.1,0.5,1, 5, 10]  	   # regr的超参数
	l1_ratio_grid = [0, 0.2, 0.5, 0.7, 1]  # regr的超参数
	
	param_grid = {
		'selector__k': k_grid,
		'selector__score_func': score_func_grid,
		'regr__alpha': alpha_grid,
		'regr__l1_ratio': l1_ratio_grid,
	}
	
	inner_cv = model_selection.RepeatedKFold(n_splits=5, n_repeats=5)
	mcc_scorer = metrics.make_scorer(metrics.r2_score, greater_is_better=True)
	
	print("超参数包括:{}".format(estimator.get_params().keys()))
	
	
	try:
		search = joblib.load(grid_search_file)
  		print("网格搜索文件已导入")
	except FileNotFoundError:
		search = model_selection.GridSearchCV(
			estimator=estimator, param_grid=param_grid,
			cv=inner_cv, scoring=mcc_scorer,n_jobs=-1,
			refit=True, return_train_score=False  # return_train_score=True computationally expensive
		)
	
		search.fit(x_train, y_train)
		joblib.dump(search, grid_search_file, compress=3)
		print("网格搜索文件已保存")
	
	print("网格搜索最优超参数:{}".format(search.best_params_))
	print("网格搜索最优性能:{}".format(search.best_score_))
	
	estimator.set_params(**search.best_params_)
	estimator.fit(x_train, y_train)
	y_train_predicted = estimator.predict(x_train)
 
	r,r2,mae,mse,rmse=myMetrics(y_train,y_train_predicted)
	print("训练集评估:R:{},R2:{},MAE:{},MSE:{},RMSE:{}".format(r,r2,,ae,mse,rmse))

	mask = np.asarray(estimator.named_steps['selector'].get_support())
	n_selected_features = sum(mask)
	n_features = x_train.shape[1]
	
	print('特征选择:{}/{}'.format(n_selected_features, n_features))
	
	featureName = np.asarray(featureName)
	
	print("未被选择的特征:{}".format(featureName[mask == False]))
	
	y_predicted = estimator.predict(x_test)
	#print("y_predicted={}".format(y_predicted))
	r,r2,mae,mse,rmse=myMetrics(y_test,y_predicted)
	print("测试集评估:R:{},R2:{},MAE:{},MSE:{},RMSE:{}".format(r,r2,,ae,mse,rmse))
	
	models = []
	
	for i, (train_index, test_index) in enumerate(inner_cv.split(x_train)):
		estimator = deepcopy(estimator)
		estimator.set_params(**search.best_params_)
		estimator.fit(x_train[train_index], y_train[train_index])
		models.append(estimator)
	
	joblib.dump(models, model_file, compress=3)
	print("模型文件已保存")


In [19]:
myElasticNet(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test,featureName=featureName,k_min=50,k_step=100,model_file=modelPath+path+'.pkl',grid_search_file=modelPath+path+'_search.pkl')

超参数包括:dict_keys(['memory', 'steps', 'verbose', 'stand_scaler', 'minmax_scaler', 'selector', 'regr', 'stand_scaler__copy', 'stand_scaler__with_mean', 'stand_scaler__with_std', 'minmax_scaler__clip', 'minmax_scaler__copy', 'minmax_scaler__feature_range', 'selector__k', 'selector__score_func', 'regr__alpha', 'regr__copy_X', 'regr__fit_intercept', 'regr__l1_ratio', 'regr__max_iter', 'regr__normalize', 'regr__positive', 'regr__precompute', 'regr__random_state', 'regr__selection', 'regr__tol', 'regr__warm_start'])
网格搜索最优超参数:{'regr__alpha': 1, 'regr__l1_ratio': 0.5, 'selector__k': 300, 'selector__score_func': <function mutual_info_regression at 0x7f7b94c8b550>}
网格搜索最优性能:-2051242.8415625


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


训练集r2:0.6386423327945794
特征选择:300/765
未被选择的特征:['acetylation' 'modlAMPCharge' 'modlAMPpI' 'PymoleculeLDI' 'PymoleculeRnc'
 'PymoleculeRpc' 'PymoleculeMac' 'PymoleculeMnc' 'PymoleculeMpc'
 'PymoleculeQass' 'PymoleculeQHss' 'PymoleculeQmax' 'PymoleculeQOmin'
 'PymoleculeQNmin' 'PymoleculeQCmin' 'PymoleculeQHmin' 'PymoleculeQOmax'
 'PymoleculeQHmax' 'PymoleculemChi1' 'PymoleculeChi4c' 'PymoleculeChi3ch'
 'PymoleculeChi4ch' 'PymoleculeChi6ch' 'Pymoleculeknotp' 'PymoleculeChiv9'
 'PymoleculeChiv4c' 'PymoleculeChiv3ch' 'PymoleculeChiv4ch'
 'PymoleculeChiv6ch' 'Pymoleculeknotpv' 'PymoleculeS1' 'PymoleculeS2'
 'PymoleculeS3' 'PymoleculeS4' 'PymoleculeS5' 'PymoleculeS6'
 'PymoleculeS8' 'PymoleculeS10' 'PymoleculeS11' 'PymoleculeS14'
 'PymoleculeS15' 'PymoleculeS18' 'PymoleculeS19' 'PymoleculeS20'
 'PymoleculeS22' 'PymoleculeS24' 'PymoleculeS26' 'PymoleculeS28'
 'PymoleculeS29' 'PymoleculeS30' 'PymoleculeS31' 'PymoleculeS32'
 'PymoleculeS33' 'PymoleculeS34' 'PymoleculeS36' 'PymoleculeS37'
 'Pymol

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fold 1:
Train:index=[ 0  1  2  3  5  6  8  9 10 11 12 14 15 16 17 19 21 22 24 25 26 28 29 30
 31 32 33 34 35 36 37 39 40 42]
Test:index=[ 4  7 13 18 20 23 27 38 41]


...

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fold 23:
Train:index=[ 0  2  3  4  5  6  7  8 10 11 12 13 14 17 19 20 21 22 23 24 25 26 28 29
 30 31 33 35 36 37 38 39 40 41 42]
Test:index=[ 1  9 15 16 18 27 32 34]


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fold 24:
Train:index=[ 0  1  2  4  7  8  9 10 13 14 15 16 17 18 19 20 21 22 23 26 27 28 29 30
 31 32 33 34 35 36 37 38 39 41 42]
Test:index=[ 3  5  6 11 12 24 25 40]


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
end = time.perf_counter()
print("目前运行时间为", round(end-start), 'seconds')

# Human blood (nature)

In [ ]:
Human_blood_nature,featureName=readFile(rootPath+'Human blood (nature).xlsx')

In [ ]:
Human_blood_nature=Dataset(Human_blood_nature,featureName,['label'],cut_len=3, max_len=50, min_len=0)
x_train, y_train, x_test, y_test=Human_blood_nature.getData()

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

# mouse blood (modification)

In [ ]:
mouse_blood_modification,featureName=readFile(rootPath+'mouse blood（modification）.xlsx')

In [ ]:
mouse_blood_modification=Dataset(mouse_blood_modification,featureName,['label'],cut_len=3, max_len=50, min_len=0)
x_train, y_train, x_test, y_test=mouse_blood_modification.getData()

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

# mouse blood (nature)

In [ ]:
mouse_blood_nature,featureName=readFile(rootPath+'mouse blood（nature）.xlsx')

In [ ]:
mouse_blood_nature=Dataset(mouse_blood_nature,featureName,['label'],cut_len=3, max_len=50, min_len=0)
x_train, y_train, x_test, y_test=mouse_blood_nature.getData()

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

# mouse crude intestinal (modification)

In [ ]:
mouse_intestinal_modification,featureName=readFile(rootPath+'mouse crude intestinal (modification).xlsx')

In [ ]:
mouse_intestinal_modification=Dataset(mouse_intestinal_modification,featureName,['label'],cut_len=3, max_len=50, min_len=0)
x_train, y_train, x_test, y_test=mouse_intestinal_modification.getData()

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape